# Additional Modelling Challenges

## Traffic Lights

Consider a four-way traffic junction with eight traffic lights.

- Four of the traffic lights are for the vehicles and can be represented by the variables V1 to V4 with domains {r,g,y} (for red, green and yellow).
- The other four traffic lights are for the pedestrians and can be represented by the variables P1 to P4 with domains {r,g}.

Model the problem and find all solutions such that no collisions occur, and the amount of green lights is maximized.

In [1]:
import cpmpy as cp

# Parameters
r, g, y = 0, 1, 2
n_periods = 3 * 4

# Decision variables
v1, v2, v3, v4 = cp.intvar(r, y, shape=(4, n_periods))
p1, p2, p3, p4 = cp.intvar(r, g, shape=(4, n_periods))
traffic_lights = [v1, p1, v2, p2, v3, p3, v4, p4]

model = cp.Model()

# Constraints
for t in range(n_periods):
    # North-South vs East-West vehicle traffic
    model += ~((v1[t] == g) & (v2[t] == g))
    model += ~((v1[t] == g) & (v4[t] == g))
    model += ~((v3[t] == g) & (v2[t] == g))
    model += ~((v3[t] == g) & (v4[t] == g))
    
    # Pedestrians cannot cross when perpendicular traffic is green or yellow
    model += ~((p1[t] == g) & ((v1[t] == g) | (v1[t] == y)))
    model += ~((p1[t] == g) & ((v3[t] == g) | (v3[t] == y)))
    model += ~((p2[t] == g) & ((v2[t] == g) | (v2[t] == y)))
    model += ~((p2[t] == g) & ((v4[t] == g) | (v4[t] == y)))
    model += ~((p3[t] == g) & ((v3[t] == g) | (v3[t] == y)))
    model += ~((p3[t] == g) & ((v1[t] == g) | (v1[t] == y)))
    model += ~((p4[t] == g) & ((v4[t] == g) | (v4[t] == y)))
    model += ~((p4[t] == g) & ((v2[t] == g) | (v2[t] == y)))

    # Transitions
    if t > 0:
        for v in [v1, v2, v3, v4]:
            model += v[t] == ((v[t-1] + 1) % 3)

model.maximize(cp.sum([cp.Count(tl, g) for tl in traffic_lights]))

if model.solve():
    print("Solution found!")
    for tl in traffic_lights:
        print(tl.value())
else:
    print("No solution found.")

Solution found!
[1 2 0 1 2 0 1 2 0 1 2 0]
[0 0 1 0 0 1 0 0 1 0 0 1]
[0 1 2 0 1 2 0 1 2 0 1 2]
[1 0 0 1 0 0 1 0 0 1 0 0]
[1 2 0 1 2 0 1 2 0 1 2 0]
[0 0 1 0 0 1 0 0 1 0 0 1]
[0 1 2 0 1 2 0 1 2 0 1 2]
[1 0 0 1 0 0 1 0 0 1 0 0]
